# Наивный байесовский классификатор для фильтрации спама

В этом ноутбуке реализован наивный байесовский классификатор для определения спам-писем.

## Импорт библиотек

In [1]:
import pandas as pd
import numpy as np

## Загрузка данных

Загружаем датасет с электронными письмами:

In [2]:
emails = pd.read_csv('data/emails.csv')
emails

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
...,...,...
5723,Subject: re : research and development charges...,0
5724,"Subject: re : receipts from visit jim , than...",0
5725,Subject: re : enron case study update wow ! a...,0
5726,"Subject: re : interest david , please , call...",0


## Предварительная обработка данных

Определим функцию для обработки текста писем:

In [3]:
def process_email(text):
    """Приводит текст к нижнему регистру и возвращает уникальные слова"""
    text = text.lower()
    return list(set(text.split()))

Применяем функцию к данным:

In [4]:
emails['words'] = emails['text'].apply(process_email)
emails

,text,spam,words
0,Subject: naturally irresistible your corporate...,1,"[., without, -, original, ordered, list, marke..."
1,Subject: the stock trading gunslinger fanny i...,1,"[plain, gunslinger, try, the, chronography, di..."
2,Subject: unbelievable new homes made easy im ...,1,"[,, all, the, approved, loan, -, ., subject:, ..."
3,Subject: 4 color printing special request add...,1,"[advertisement, ,, click, ca, azusa, 626, ), ...."
4,"Subject: do not have money , get software cds ...",1,"[,, all, death, be, the, by, ., subject:, grea..."
...,...,...,...
5723,Subject: re : research and development charges...,0,"[gpg, i, locate, watson, ., -, 7, thanks, vera..."
5724,"Subject: re : receipts from visit jim , than...",0,"[i, ., -, tomorrow, thanks, louisiana, 02, /, ..."
5725,Subject: re : enron case study update wow ! a...,0,"[i, document, ., -, attempt, then, tx, thanks,..."
5726,"Subject: re : interest david , please , call...",0,"[i, ., -, discuss, thanks, about, /, kaminski,..."


## Поиск априорных вероятностей

Сначала найдем вероятность того, что электронное письмо — спам (априорную).  
Для этого подсчитаем количество писем, которые являются спамом, и разделим его на общее их количество. 

In [5]:
print(f'Вероятность того, что электронное письмо является спамом, когда мы ничего не знаем о нем: {sum(emails['spam']) / len(emails):.0%}')
print(f'Аналогично, априорная вероятность того, что электронное письмо полезное: {1 - sum(emails['spam']) / len(emails):.0%}')

Вероятность того, что электронное письмо является спамом, когда мы ничего не знаем о нем: 24%
Аналогично, априорная вероятность того, что электронное письмо полезное: 76%


# Нахождение апостериорной вероятности с помощью теоремы Байеса

Нам нужно найти вероятность того, что спам (и полезные) электронные письма содержат определенное слово.  
Проделаем это для всех слов одновременно.  
Следующая функция создает словарь model, который записывает каждое слово рядом с количеством его появлений в спам- и полезных письмах:

In [6]:
model = {}

for index, email in emails.iterrows():
    for word in email['words']:
        # Инициализация словаря для нового слова
        if word not in model:
            model[word] = {'spam': 1, 'ham': 1} # Добавляем 1 для сглаживания
        if word in model:
            # Обновляем счетчики
            if email['spam']:
                model[word]['spam'] += 1
            else:
                model[word]['ham'] += 1

In [7]:
print(model['lottery'])
print(model['sale'])

{'spam': 9, 'ham': 1}
{'spam': 39, 'ham': 42}


Это означает, что слово «лотерея» появляется в одном полезном электронном письме и девяти спам-письмах, в то время как слово «распродажа» —
в 42 полезных электронных письмах и 39 спам-письмах.  
Хотя этот словарь не содержит никаких вероятностей, их можно вывести, разделив первую запись на сумму обеих записей.

In [8]:
print(f'Таким образом, если электронное письмо содержит слово «лотерея» ("lottery"), вероятность \
того, что оно является спамом, равна 9 / (9 + 1) = {9 / (9 + 1):.1%} \
\nА если в нем есть слово «распродажа» ("sale"), то 39 / (39 + 42) = {39/(39+ 42):.2%}')

Таким образом, если электронное письмо содержит слово «лотерея» ("lottery"), вероятность того, что оно является спамом, равна 9 / (9 + 1) = 90.0% 
А если в нем есть слово «распродажа» ("sale"), то 39 / (39 + 42) = 48.15%


# Реализация наивного байесовского алгоритма

In [9]:
def predict_naive_bayes(email):
    """
    Предсказывает вероятность того, что письмо является спамом
    с помощью наивного байесовского классификатора.
    """
    # вычисляем общее кол-во электронных писем, нежелательных и полезных
    total = len(emails)
    num_spam = sum(emails['spam'])
    num_ham = total - num_spam

    # обрабатываем каждое электронное письмо, превращая его в список слов в нижнем регистре
    email = email.lower()
    words = set(email.split())
    
    # Инициализация вероятностей
    spams = [1.0]  # Начальное значение для умножения
    hams = [1.0]

    for word in words:
        if word in model:

            # для каждого слова вычисляем условную вероятность того, что 
            # содержащее его электронное письмо является спамом (или полезным)
            spams.append(model[word]['spam'] / num_spam * total)
            hams.append(model[word]['ham'] / num_ham * total)
            
    # Расчет произведения вероятностей в логарифмической шкале
    # перемножаем все предыдущие вероятности, умноженные на априорную вероятность того,
    # что электронное письмо является спамом, и называем всё это prod_spams
    # выполняем аналогичный процесс для prod_hams
    prod_spams = float(np.prod(spams) * num_spam)
    prod_hams = float(np.prod(hams) * num_ham)

    # нормализуем эти две вероятности, чтобы они составляли единицу 
    # (с помощью теоремы Байеса), и возвращаем результат
    return prod_spams / (prod_spams + prod_hams)

## Тестирование модели

Протестируем модель на различных примерах:

In [10]:
email_texts = [
    'lottery sale',
    'Hi mom how are you',
    'Hi MOM how aRe yoU afdjsaklfsdhgjasdhfjklsd',
    'meet me at the lobby of the hotel at nine am',
    'enter the lottery to win three million dollars',
    'buy cheap lottery easy money now',
    'Grokking Machine Learning by Luis Serrano',
    'asdfgh'
]

print(f"{'Текст письма':^50} | {'Вероятность спама'}")
print('-' * 70)

for email_text in email_texts:
    probability = predict_naive_bayes(email_text)
    print(f"{email_text:<50} | {probability:>7.2%}")

                   Текст письма                    | Вероятность спама
----------------------------------------------------------------------
lottery sale                                       |  96.38%
Hi mom how are you                                 |  12.55%
Hi MOM how aRe yoU afdjsaklfsdhgjasdhfjklsd        |  12.55%
meet me at the lobby of the hotel at nine am       |   0.01%
enter the lottery to win three million dollars     |  99.95%
buy cheap lottery easy money now                   | 100.00%
Grokking Machine Learning by Luis Serrano          |  41.97%
asdfgh                                             |  23.88%
